In [1]:
import requests
import re
from bs4 import BeautifulSoup
from collections import defaultdict
#from progressbar import *
import time
import json

# Example of question and answer (answer is html)

examplePage = requests.get("http://student.uva.nl/en/faq/application-and-admission/application/application.html")

exampleSoup = BeautifulSoup(examplePage.content, 'html.parser')

for faqItem in exampleSoup.find_all('div', class_='faq-item'):
    
    # plain text
    question = faqItem.find_all('a', class_='faq-question')[0].get_text()
    print(question)
    
    # html
    answer = faqItem.find_all('div', class_='faq-answer')[0]
    print(answer.prettify())
    break
    

What Bachelor’s programmes are offered by the UvA?
<div class="faq-answer">
 <p>
  An overview of all our Bachelor’s programmes can be found online. On each degree programme page, you will find information about the relevant degree programme, admission requirements, deadlines and contact details.
 </p>
 <ul>
  <li>
   <a href="http://www.uva.nl/en/education/bachelor-s/bachelors.html">
    Bachelor’s programmes
   </a>
   <br/>
  </li>
 </ul>
</div>



In [2]:
# WITH PROGRESSBAR

# Using depth-bounded depth-first search for finding urls with FAQs
# (not really object-oriented)

# A nice progress bar, since this will take a while.
# Does require global variables ad makes the code 
# less nice in other ways as well.
#widgets = ['Test: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
 #      ' ', ETA()] 

def faq_list_generator_pb(startPage, depth, pagesRegex):
    # so you don't do pages twice
    crawled = set()
    #global pbar
    #pbar.start()
    
    faqList = depth_first_urls_pb(startPage, depth, crawled, pagesRegex)
    
    #pbar.finish()
    return faqList

def depth_first_urls_pb(currentPage, depth, crawled, pagesRegex):
    faqList = []
    
    currentSoup = BeautifulSoup(currentPage.content, 'html.parser')
    
    # no need to add anything if it's already added, but we still might need what's underneath
    if currentPage.url not in crawled:
        # construct list for current page
        for faqItem in currentSoup.find_all('div', class_='faq-item'):
            faqList.append((faqItem, currentPage.url))
    
    crawled.add(currentPage.url)
        
    # continue deeper
    if depth > 0:
        # for each page under this page
        for i, soupTag in enumerate(currentSoup.findAll('a', attrs={'href': re.compile(pagesRegex)})):
           #global maxDepth
            #if depth == maxDepth:
                #global pbar
                #pbar.update(i)
            link = soupTag.get('href')
            if link[0] == '/':
                if "student.uva.nl" in currentPage.url:
                    link = "http://student.uva.nl" + link
                elif "uva.nl" in currentPage.url:
                    link = "http://www.uva.nl" + link
                    
            nextPage = None
            try:
                nextPage = requests.get(link)
            except:
                print("This link doesn't look right:")
                print(link)
                continue
                
            # add whatever comes out of this link and what's below it
            faqList = faqList + depth_first_urls_pb(nextPage, depth - 1, crawled, pagesRegex)
                
    return faqList

In [3]:
# WITHOUT PROGRESSBAR

# Using depth-bounded depth-first search for finding urls with FAQs
# (not really object-oriented)

# Change according to current search criteria
# pagesRegex = "^http://www.student.uva.nl|^http://student.uva.nl|^http://www.uva.nl|^http://uva.nl|^/"
pagesRegex = "^/"

def faq_list_generator(startPage, depth, pagesRegex):
    # so you don't do pages twice
    crawled = set()
    faqList = depth_first_urls(startPage, depth, crawled, pagesRegex)
    return faqList

def depth_first_urls(currentPage, depth, crawled, pagesRegex):
    faqList = []
    
    currentSoup = BeautifulSoup(currentPage.content, 'html.parser')
    
    # no need to add anything if it's already added, but we still might need what's underneath
    if currentPage.url not in crawled:
        # construct list for current page
        for faqItem in currentSoup.find_all('div', class_='faq-item'):
            faqList.append((faqItem, currentPage.url))
    
    crawled.add(currentPage.url)
        
    # continue deeper
    if depth > 0:
        # for each page under this page
        for soupTag in currentSoup.findAll('a', attrs={'href': re.compile(pagesRegex)}):
            link = soupTag.get('href')
            if link[0] == '/':
                if "student.uva.nl" in currentPage.url:
                    link = "http://student.uva.nl" + link
                elif "uva.nl" in currentPage.url:
                    link = "http://www.uva.nl" + link
                    
            nextPage = None
            try:
                nextPage = requests.get(link)
            except:
                print("This link doesn't look right:")
                print(link)
                continue
                
            # add whatever comes out of this link and what's below it
            faqList = faqList + depth_first_urls(nextPage, depth - 1, crawled, pagesRegex)
                
    return faqList

# CHANGE according to your needs
depth = 2
startPage = requests.get("http://student.uva.nl/veelgestelde-vragen/veelgestelde-vragen.html")
pagesRegex = "^/"

faqList = faq_list_generator(startPage, depth, pagesRegex)

In [8]:
# CHANGE according to your needs
maxDepth = 3
# startPage = requests.get("http://student.uva.nl/veelgestelde-vragen/veelgestelde-vragen.html")

startPage = requests.get("http://student.uva.nl/veelgestelde-vragen/veelgestelde-vragen.html")
# you can increase tree width by adding more options for the start of the url
pagesRegex = "^/"


faqList = faq_list_generator_pb(startPage, maxDepth, pagesRegex)
# OR without progressbar
# faqList = faq_list_generator(startPage, maxDepth, pagesRegex)


# ADD MORE IF YOU WANT
# Note that maxDepth is 1 if you just want the page itself
# faqList = faqList + faq_list_generator(startPage2, maxDepth2, pagesRegex2)
# etc.

In [ ]:
# startPage = requests.get("http://www.uva.nl/home")
# startPage = requests.get("http://student.uva.nl/veelgestelde-vragen/ict-en-faciliteiten/ict-en-faciliteiten.html")
# startPage = requests.get("http://student.uva.nl/veelgestelde-vragen/ict-en-faciliteiten/blackboard/blackboard.html")
# startPage = requests.get("http://student.uva.nl/content/az/blackboard/veelgestelde-vragen/veelgestelde-vragen.html")
# pagesRegex = "^http://www.student.uva.nl|^http://student.uva.nl|^http://www.uva.nl|^http://uva.nl|^/"

In [ ]:
import nltk

stopwords = {}
dutch_stopwords = set()
with open('../stopwoorden.txt') as f:
    for line in f:
        dutch_stopwords.add(line)
nltk_dutch = set(nltk.corpus.stopwords.words('dutch'))
stopwords['dutch'] = dutch_stopwords.union(nltk_dutch)
stopwords['english'] = set(nltk.corpus.stopwords.words('english'))

def generate_keywords(sentence, lang=(1, "English")):
    tokens = {'NOUN', 'ADJ', 'NUM'}
    l = lang[1].lower()
    sentence = sentence.lower()
    tokenized = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag([x for x in tokenized if x not in stopwords[l]],
                          tagset='universal', lang=l)
    return  ','.join(str(s) for s in set([word for (word, token) in tagged if token in tokens]))


In [9]:
qAndA = {}
for (x, _) in faqList:
    question = x.find_all('a', class_='faq-question')[0].get_text()
    answer = str(x.find_all('div', class_='faq-answer')[0])
    #qAndA[generate_keywords(question)] = (question, answer)
    qAndA[question] = answer
    
# Possible extension:
# It's also possible to use the urls in faqList, e.g.
# for (_, url) in faqList:

# And now you can use this dictionary of questions and answers
print(len(qAndA))

177


In [10]:
# EXPORTING
with open('export_faq_nl_nokw.txt', 'w') as file:
     file.write(json.dumps(qAndA))